In [55]:
%%sh
ls /home/glue_user/data

bru.csv
vlg.csv
wal.csv


In [56]:
%%sh
ls /home/glue_user/contract

addresses.sol


In [57]:
# Configure connection with node
from web3 import Web3
w3 = Web3(Web3.HTTPProvider('http://ethnode:8545'))

In [58]:
# Smoke test connection
print(w3.eth.get_block('latest'))

AttributeDict({'number': 0, 'hash': HexBytes('0x18ff4a4dcbc85213fdb4ed1022f7d8a196d20ec8d2cb520b1367fd61db72d360'), 'parentHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'), 'mixHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'), 'nonce': HexBytes('0x0000000000000000'), 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'), 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), 'transactionsRoot': Hex

In [59]:
# Configure default account used to sign transactions
w3.eth.default_account = w3.eth.accounts[0]
print(w3.eth.accounts)
print(w3.eth.default_account)

['0x70157348Ff5cb951094e00F8B52cF26314ac23e5', '0x5348A398bF0BE37Bc45aBdc2Ec8F1FD34EfC7332', '0xD4928Cb9A2F7ecCF5f4a9363B75Dc6519ECC6811', '0xC87324A7C8A6F2B8B0247d411604da1B6A0ABd08', '0x582464e8Fd01665347d67B388C943e47fA6152f1', '0xc2b51c4B53A1326D7b2C34b111eA06073Ce80db5', '0xE55f10C681E6dbBe1052d48fC88571f000258527', '0x4887Ba67066aF78D7705a0209Bdc8f78eAC54Ef0', '0x341FAAF66840680fbD664390ED1abc0bf067123E', '0x4656853523f01f023e51d4836e6a4d633ce31CAc']
0x70157348Ff5cb951094e00F8B52cF26314ac23e5


In [60]:
# Configure smart contract compiler
from solcx import install_solc
install_solc(version='latest')

Version('0.8.15')

In [61]:
# Read contract source code
from solcx import compile_source
with open('/home/glue_user/contract/addresses.sol') as f:
    compiled_sol = compile_source(f.read(), output_values=['abi', 'bin'])

In [62]:
# Extract smart contract abi and bytecode
contract_id, contract_interface = compiled_sol.popitem()
bytecode = contract_interface['bin']
abi = contract_interface['abi']

In [63]:
# Deploy smart contract
addresses_contract = w3.eth.contract(abi=abi, bytecode=bytecode)
tx_hash = addresses_contract.constructor().transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
deployed_addresses_contract = w3.eth.contract(address=tx_receipt.contractAddress,abi=abi)

In [64]:
# Smoke test : call a contract view function
deployed_addresses_contract.functions.listPostcodes().call()

[]

In [65]:
# Add addresses
import csv
with open('/home/glue_user/data/bru.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for address in reader:
        tx_hashes = []
        if address["streetname_de"] != '':
            # A DE version of the address is available - Add it to ledger
            tx_hashes.append(deployed_addresses_contract.functions.createAddress(
                address["streetname_de"],
                str(address["postcode"]),
                str(address["house_number"]),
                str(address["box_number"]),
                str(address["EPSG:4326_lat"]),
                str(address["EPSG:4326_lon"])
            ).transact())
        if address["streetname_nl"] != '':
            # A NL version of the address is available - Add it to ledger
            tx_hashes.append(deployed_addresses_contract.functions.createAddress(
                address["streetname_nl"],
                str(address["postcode"]),
                str(address["house_number"]),
                str(address["box_number"]),
                str(address["EPSG:4326_lat"]),
                str(address["EPSG:4326_lon"])
            ).transact())
        if address["streetname_fr"] != '':
            # A FR version of the address is available - Add it to ledger
            tx_hashes.append(deployed_addresses_contract.functions.createAddress(
                address["streetname_fr"],
                str(address["postcode"]),
                str(address["house_number"]),
                str(address["box_number"]),
                str(address["EPSG:4326_lat"]),
                str(address["EPSG:4326_lon"])
            ).transact())

    # Check transaction are complete
    address_ids = []
    for tx in tx_hashes:
        tx_r = w3.eth.wait_for_transaction_receipt(tx)
        address_ids.append(tx_r["logs"][0]["topics"][1])
    print(len(address_ids)) 
    print(address_ids[0])

ContractLogicError: execution reverted: VM Exception while processing transaction: revert Address already exist.

In [66]:
deployed_addresses_contract.functions.listPostcodes().call()

['1030',
 '1180',
 '1200',
 '1080',
 '1000',
 '1170',
 '1081',
 '1060',
 '1070',
 '1190',
 '1050',
 '1160',
 '1150',
 '1040',
 '1210',
 '1130',
 '1140',
 '1020',
 '1090',
 '1083',
 '1120',
 '1082']

In [70]:
deployed_addresses_contract.functions.listAddressIdsByPostcode("1060").call()

[b'b2\x98\xb4\x0e?\xa3\xe9\x8ed\xec%R4A\x0b/\x9b\xa4\xe5`\x04\xa5\xa9%?Nte\xb8J\xb1',
 b"\x16{)\x8cu\xf8'\xb9\xb5L\xdb\xe5J\xd1B\xe6\xfb}\xdebS\xd6\xfe\xc5s\xfe=\xe0?k\x9f\x10",
 b'\x00\xfc\x17\x0e\xc6\xc6\xc1x\xc6\x1cy\xcc\xca\xe9\xd7\x7fq\x88\x95E\x8a\x80\xb4\xcaa\x06^\x033\x13\x82\xe2',
 b'\x97\x12\xca\xc3\xa9\xaf\x11\xb3M\x06\x9ee\x0b\x850\xe3\xeb\n\x89#\x90\xae\xb5\xad\x15\xdc\x88\x99\xaf\xf0\x06R',
 b'/\x83\xf8\x12\xcf\x81\xc14Q\xb4bUg\x1fe\xd0\x19\xc5Dzb\x99#8\xbb\xd5\x8c1A\xe1:\xd6',
 b'\x11\xcdq\x1d\xa4\xf3\xbd\xde\xe5\xf6\xe4\xf7\xc60\x88\xc0\xbd\xddF\x13\xf2{\xa6c\xb7\xddg\xbdo\x1c\xacG',
 b'\xd6\x82O\x07\xf14H/e\xf3\xdb\x91\x92(\xf5\xe2\x1eF;Y\x9221\xa4e$av.+\x84\xb9',
 b'\xfd ;\xb8\x18\x81\xf1\x87\xe8H7\xff\xf3Z\x8f\x9dDEa\xd1\xb7\x0c,1\x92F\x86\x01\x17p\x9eH',
 b'\x00\x81\xc8\rY\x0fj3J\xa7\xb9\x99\xd9\xed\x81/J\xe3\xff\x86H\xa5T\xd9\x8d8\x0fP\x87\x03o\xea',
 b"?\xc73\xd3\x0c\x8e\x1a\xe3\x8d\xb3'\x02\xf7\xf0X\xbd\x17\xc1O\xc6\xd7\xe14\x19 \xc3@\x99s\x9c\xe9F",
 b'\x85\n\xb

In [68]:
deployed_addresses_contract.functions.getAddress(w3.toHex(b'\x9d$\xcek\xc91\x11\x8f\xeb\x06]\x11u<-\xed\xef\x16p\x82\xef\x15!e\x19o=\xf4\xe1\xbei\xb1')).call()

(b'\x9d$\xcek\xc91\x11\x8f\xeb\x06]\x11u<-\xed\xef\x16p\x82\xef\x15!e\x19o=\xf4\xe1\xbei\xb1',
 'Pannenhuisstraat',
 '1090',
 '29',
 '',
 '50.87556',
 '4.33857')

In [54]:
deployed_addresses_contract.functions.getAddress(w3.toHex(hexstr='0x5f0bc48913d26ec45832b1b4a20620c4b5b8ee0e8878fde6d2a464e9424c1763')).call()

(b'_\x0b\xc4\x89\x13\xd2n\xc4X2\xb1\xb4\xa2\x06 \xc4\xb5\xb8\xee\x0e\x88x\xfd\xe6\xd2\xa4d\xe9BL\x17c',
 'Rue Général Gratry',
 '1030',
 '49',
 '',
 '50.84613',
 '4.39996')

In [9]:
# Read addresses data
addresses = spark.read.option("header",True).csv("/home/glue_user/data")
addresses.show()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+-------------+-------------+----------+----------+------------+---------------+--------------------+--------------------+--------------------+--------+---------------+-----------+---------+-------------+--------------------+-------------+-----------+-------+
|EPSG:31370_x|EPSG:31370_y|EPSG:4326_lat|EPSG:4326_lon|address_id|box_number|house_number|municipality_id|municipality_name_de|municipality_name_fr|municipality_name_nl|postcode|    postname_fr|postname_nl|street_id|streetname_de|       streetname_fr|streetname_nl|region_code| status|
+------------+------------+-------------+-------------+----------+----------+------------+---------------+--------------------+--------------------+--------------------+--------+---------------+-----------+---------+-------------+--------------------+-------------+-----------+-------+
|150915.63200|152830.76200|     50.68606|      4.38171|   1522363|      B001|          93|          25014|                null|     Braine-l'A